In [ ]:
"""Steps:
1) Run the cells one by one in order
2) Create a folder called YOLO_IITISOC in the 'My Drive' section of Google Drive
3) Upload the training images as YOLO_TrainImages.zip and testing images as YOLO_TestImages.zip in the folder created in step 2 
4) When u run the cell where it connects to drive, click on the link that appears->choose the google account->copy and paste the authentication code
5) Run all the other cells now 
"""

In [ ]:
"""Anything appearing after the exclamation point on a line will be executed not by the Python kernel, but by the system command line"""

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive  #This is to make a short form for the mydrive location 
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'  'MobileNet_Project (23-06-2021).zip'   YOLO_IITISOC


In [ ]:
# Clone the darknet repository from GitHub as a part of Transfer Learning
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Configure some settings
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
# Compile
!make

In [ ]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Change some details in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg 
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg

In [ ]:
!echo -e 'With_Mask\nWithout_Mask' > data/face_mask.names
!echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/face_mask.names\nbackup = /mydrive/YOLO_IITISOC' > data/face_mask.data

In [ ]:
!cp cfg/yolov3_training.cfg /mydrive/YOLO_IITISOC/yolov3_testing.cfg
!cp data/face_mask.names /mydrive/YOLO_IITISOC/classes.txt

In [ ]:
!mkdir data/face_mask_train_img
!unzip /mydrive/YOLO_IITISOC/YOLO_TrainImages.zip -d data/face_mask_train_img

In [ ]:
!mkdir data/face_mask_test_img
!unzip /mydrive/YOLO_IITISOC/YOLO_TestImages.zip -d data/face_mask_test_img

In [ ]:
import glob
images_list = glob.glob("data/face_mask_train_img/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

In [ ]:
images_list = glob.glob("data/face_mask_test_img/*.jpg")
with open("data/test.txt", "w") as f:
    f.write("\n".join(images_list))

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
!chmod +x ./darknet #This is to deal with the permissions

In [ ]:
#Training
!./darknet detector train data/face_mask.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show -map 
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/face_mask.data cfg/yolov3_training.cfg /mydrive/YOLO_IITISOC/yolov3_training_last.weights -dont_show -map

In [ ]:
"""
Now go to YOLO_IITI folder in MyDrive and download:
1) classes.txt
2) yolov3_testing.cfg 
3) yolov3_training_last.weights
"""

In [ ]:
!./darknet detector map data/face_mask.data cfg/yolov3_training.cfg /mydrive/YOLO_IITISOC/yolov3_training_last.weights

In [ ]:
!./darknet detector test data/face_mask.data cfg/yolov3_training.cfg /mydrive/YOLO_IITISOC/yolov3_training_last.weights

In [ ]:
!pwd

In [ ]:
!./darknet detector demo data/face_mask.data cfg/yolov3_training.cfg /mydrive/YOLO_IITISOC/yolov3_training_last.weights -dont_show test_video.mp4 -i 0 -out_filename result_video.avi -thresh 0.7